In [24]:
!pip install pyspark

In [25]:
print('Displaying the first 5 lines of the android.log file:')
!head -n 5 /content/Android.log

Displaying the first 5 lines of the android.log file:
12-17 19:31:36.263  1795  1825 I PowerManager_screenOn: DisplayPowerStatesetColorFadeLevel: level=1.0
12-17 19:31:36.263  5224  5283 I SendBroadcastPermission: action:android.com.huawei.bone.NOTIFY_SPORT_DATA, mPermissionType:0
12-17 19:31:36.264  1795  1825 D DisplayPowerController: Animating brightness: target=21, rate=40
12-17 19:31:36.264  1795  1825 I PowerManager_screenOn: DisplayPowerController updatePowerState mPendingRequestLocked=policy=BRIGHT, useProximitySensor=true, useProximitySensorbyPhone=true, screenBrightness=33, screenAutoBrightnessAdjustment=0.0, brightnessSetByUser=true, useAutoBrightness=true, blockScreenOn=false, lowPowerMode=false, boostScreenBrightness=false, dozeScreenBrightness=-1, dozeScreenState=UNKNOWN, useTwilight=false, useSmartBacklight=true, brightnessWaitMode=false, brightnessWaitRet=true, screenAutoBrightness=-1, userId=0
12-17 19:31:36.264  1795  2750 I PowerManager_screenOn: DisplayPowerState Up

In [26]:
from datetime import datetime

############
#by_chatgpt#
############

# Definition of extract_minute_from_log_line (from previous unexecuted cell 2f6db62e)
def extract_minute_from_log_line(log_line):
    try:
        timestamp_str_full = log_line.split(' ', 2)[0] + ' ' + log_line.split(' ', 2)[1]

        # Extract 'MM-DD HH:MM'
        month_day_time_str = timestamp_str_full.split(' ')[0] + ' ' + timestamp_str_full.split(' ')[1].rsplit(':', 1)[0]

        # Prepend a placeholder year (e.g., '2023-')
        full_timestamp_str = f'2023-{month_day_time_str}'

        # Parse into a datetime object
        dt_object = datetime.strptime(full_timestamp_str, '%Y-%m-%d %H:%M')

        # Format to 'YYYY-MM-DD HH:MM'
        return dt_object.strftime('%Y-%m-%d %H:%M')
    except (IndexError, ValueError) as e:
        # Handle cases where log line might not conform to expected format
        return None

In [27]:
from pyspark import SparkContext

try:
  sc.stop()
except:
  pass
sc = SparkContext(appName='countLogsDistribué')
log_RDD = sc.textFile("Android.log")

minutes_rdd = log_RDD.map(extract_minute_from_log_line).filter(lambda x: x is not None)


minute_pairs_rdd = minutes_rdd.map(lambda minute: (minute, 1))

logs_per_minute_rdd = minute_pairs_rdd.reduceByKey(lambda a, b: a + b)

print("Successfully transformed log_RDD into")

Successfully transformed log_RDD into


In [28]:
sorted_logs_per_minute_rdd = logs_per_minute_rdd.sortByKey()
collected_logs_per_minute = sorted_logs_per_minute_rdd.collect()
for entry in collected_logs_per_minute[:10]:
    print(entry)

('2023-12-17 19:31', 11782)
('2023-12-17 19:32', 6021)
('2023-12-17 19:33', 9429)
('2023-12-17 19:34', 3799)
('2023-12-17 19:35', 5580)
('2023-12-17 19:36', 955)
('2023-12-17 19:37', 204)
('2023-12-17 19:40', 3769)
('2023-12-17 19:41', 93)
('2023-12-17 19:42', 2207)


In [13]:
out_dir = "sorted_logs_per_minute"
sorted_logs_per_minute_rdd.saveAsTextFile(out_dir)

Successfully saved sorted log counts per minute to directory: sorted_logs_per_minute


In [29]:
from datetime import datetime

############
#by_chatgpt#
############

def extract_hour_from_log_line(log_line):
    try:
        timestamp_str_full = log_line.split(' ', 2)[0] + ' ' + log_line.split(' ', 2)[1]
        # Extract 'MM-DD HH'
        month_day_time_str = timestamp_str_full.split(' ')[0] + ' ' + timestamp_str_full.split(' ')[1].split(':')[0]

        # Prepend a placeholder year (e.g., '2023-')
        full_timestamp_str = f'2023-{month_day_time_str}'

        # Parse into a datetime object
        dt_object = datetime.strptime(full_timestamp_str, '%Y-%m-%d %H')

        # Format to 'YYYY-MM-DD HH'
        return dt_object.strftime('%Y-%m-%d %H')
    except (IndexError, ValueError) as e:
        return None



In [30]:
hour_rdd = log_RDD.map(extract_hour_from_log_line).filter(lambda x: x is not None)
hour_counts_rdd = hour_rdd.map(lambda hour: (hour, 1)).reduceByKey(lambda a, b: a + b)
sorted_hour_counts = hour_counts_rdd.sortByKey().collect()

print("Log counts per hour:")
for entry in sorted_hour_counts:
    print(entry)


Log counts per hour:
('2023-12-17 19', 77059)
('2023-12-17 20', 138489)
('2023-12-17 21', 198153)
('2023-12-17 22', 96650)
('2023-12-17 23', 134333)
('2023-12-18 00', 48236)
('2023-12-18 01', 3998)
('2023-12-18 02', 2436)
('2023-12-18 03', 2400)
('2023-12-18 04', 2243)
('2023-12-18 05', 2010)
('2023-12-18 06', 2366)
('2023-12-18 07', 68650)
('2023-12-18 08', 96895)
('2023-12-18 09', 58555)
('2023-12-18 10', 63337)
('2023-12-18 11', 17530)
('2023-12-18 12', 23303)
('2023-12-18 13', 7824)
('2023-12-18 14', 144006)
('2023-12-18 15', 120131)
('2023-12-18 16', 14270)
('2023-12-18 17', 77221)
('2023-12-18 18', 64810)
('2023-12-18 19', 90100)


In [36]:
out_dir_hourly = "sorted_hourly_logs"
hour_counts_rdd.saveAsTextFile(out_dir_hourly)
print(f"Successfully saved sorted hourly log counts to directory: {out_dir_hourly}")

Successfully saved sorted hourly log counts to directory: sorted_hourly_logs


In [33]:
from datetime import datetime

############
#by_chatgpt#
############

def extract_log_level_from_log_line(log_line):
    try:
        level_token = log_line.split(' ')[4]
        # Define valid log level abbreviations
        valid_levels = {'D', 'E', 'F', 'I', 'V', 'W'}
        if level_token in valid_levels:
            return level_token
        else:
            return None
    except IndexError:
        return None

In [34]:
level_rdd = log_RDD.map(extract_log_level_from_log_line).filter(lambda x: x is not None)
level_counts_rdd = level_rdd.map(lambda level: (level, 1)).reduceByKey(lambda a, b: a + b)
sorted_level_counts = level_counts_rdd.sortByKey().collect()

print("\nLog counts per level:")
for entry in sorted_level_counts:
    print(entry)


Log counts per level:
('D', 41279)
('E', 9793)
('F', 131)
('I', 39168)
('V', 6933)
('W', 4528)


In [37]:
out_dir_level = "sorted_level_logs"
level_counts_rdd.saveAsTextFile(out_dir_level)
print(f"Successfully saved sorted log counts per level to directory: {out_dir_level}")

Successfully saved sorted log counts per level to directory: sorted_level_logs
